In [ ]:
#%pip install --pre -U "weaviate-client==4.*"
#%pip install python-dotenv
# %pip install python-dateutil

In [1]:
import os
import sys

import weaviate
import weaviate.classes as wvc
import uuid
from weaviate.util import generate_uuid5
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

False

In [2]:
'''This litle code block is used anytime you want to import a local module from within a Jupyter Notebook. This is required becuase Jupyter treats each cell as a module.'''

# Navigate up one level from the current notebook's directory to reach the root directory
current_dir = os.path.dirname(os.path.realpath('__file__'))
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

In [3]:
import utils

/Users/drew_wilkins/Drews_Files/Drew/Python/ASK/.venv-311/lib/python3.11/site-packages/pydantic/_internal/_config.py:272: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
2024-02-16 13:08:41.216 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-02-16 13:08:41.234 
  command:

    streamlit run /Users/drew_wilkins/Drews_Files/Drew/Python/ASK/.venv-311/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


## 2. Set Configurations and Clients

In [ ]:
url = os.getenv("WEAVIATE_URL_COMP")
api_key = os.getenv("WEAVIATE_API_KEY_COMP")


client = weaviate.connect_to_wcs(
    cluster_url=url,
    auth_credentials=weaviate.AuthApiKey(api_key),
    headers={
        "X-OpenAI-Api-Key": os.environ.get("OPENAI_API_KEY")
    }
)

In [4]:
pdf_source_directory = "../docs/test_pdfs/"
library_catalog_directory = "../docs/library_catalog/"
zulu_format = '%Y-%m-%dT%H:%MZ'

## 2. Create the Collection and Define the Properties

In [ ]:
#create a Weaviate collection to hold PDF and PDFPages

if client.collections.exists("PDF_document"):
    client.collections.delete("PDF_document")
if client.collections.exists("PDF_document_page"):
    client.collections.delete("PDF_document_page")

pdfs_collection = client.collections.create(
    name="PDF_document",
    vectorizer_config=wvc.Configure.Vectorizer.none(),
    generative_config=wvc.Configure.Generative.openai(),
    properties=[
        wvc.Property(
            name="title",
            description="Name of the document. If none, defaults to File Name w/o extension",
            data_type=wvc.DataType.TEXT
        ),
        wvc.Property(
            name="leadership_scope",
            description="1_National, 2_District, 3_Divison, 3_Sector, 4_Flotilla, 4_Station, 5_Facility",
            data_type=wvc.DataType.TEXT
        ),
        wvc.Property(
            name="page_count",
            description="Number of pages in the document",
            data_type=wvc.DataType.INT
        ),
        wvc.Property(
            name="creation_date",
            description="Uses existing pdf date, else defaults to ingestion date",
            data_type=wvc.DataType.DATE
        ),
        wvc.Property(
            name="effective_date",
            description="Date document became effective, else defaults to ingestion date",
            data_type=wvc.DataType.DATE
        ),
            wvc.Property(
            name="tagged_date",
            description="Date document metadata was added to the library catalog",
            data_type=wvc.DataType.DATE
        ),
        wvc.Property(
            name="upsert_date",
            description="Date uploaded to the vector database",
            data_type=wvc.DataType.DATE
        ),
        wvc.Property(
            name="expiration_date",
            description="If no cancellation date given, then defaults to effective date + 10 years per COMDINST M5215.6I",
            data_type=wvc.DataType.DATE
        ),
            wvc.Property(
            name="lifecycle",
            description="Last name of Auxiliarist who curated. Currently blank",
            data_type=wvc.DataType.TEXT
            ),
        wvc.Property(
            name="aux_specific",
            description="True if document specifically applies to the Auxiliary",
            data_type=wvc.DataType.BOOL
        ),
        wvc.Property(
            name="public_release",
            description="True if document is available on public internet",
            data_type=wvc.DataType.BOOL
        ),
        wvc.Property(
            name="publication_number",
            description="Identification number of the directive or document. In the case of Directives, underscores are used for spaces (e.g., COMDTINST_M1000.6A)",
            data_type=wvc.DataType.TEXT
        ),
        wvc.Property(
            name="source",
            description="Web domain source of document (e.g.,uscg.mil, cgaux.org)",
            data_type=wvc.DataType.TEXT
        ),
        wvc.Property(
            name="organization",
            description="Not curently used, can be used to track CG directive originator using Standard Distribution List (SDL), COMDTNOTE 5605 encl (3) (i.e., CG-BSX-1) or Auxiliary Unit Number (0130510)",
            data_type=wvc.DataType.TEXT
        ),
        wvc.Property(
            name="curator",
            description="Last name of Auxiliarist who curated. Currently blank",
            data_type=wvc.DataType.TEXT
        ),
        wvc.Property(
            name="file_name",
            description="Name of the PDF file",
            data_type=wvc.DataType.TEXT
        ),
    ]
)

pdf_pages_collection = client.collections.create(
    name="PDF_document_page",
    vectorizer_config=wvc.Configure.Vectorizer.text2vec_openai(),
    generative_config=wvc.Configure.Generative.openai(),
    properties=[
        wvc.Property(
            name="content",
            description="content of the page",
            data_type=wvc.DataType.TEXT
        ),
        wvc.Property(
            name="title",
            description="Taken from PDF_document object property of the same name",
            data_type=wvc.DataType.TEXT
        ),
        wvc.Property(
            name="page_number",
            description="Page number",
            data_type=wvc.DataType.INT
        ),
        wvc.Property(
            name="publication_number",
            description="Taken from PDF_document object property of the same name",
            data_type=wvc.DataType.TEXT,
            skip_vectorization=True
        ),
    ],
    references=[
        wvc.ReferenceProperty(
            name="hasPdfDocument",
            target_collection="PDF_document"
        )
    ]
)


## Load Objects

### Option A: Load with External Source of metadata

I'm stuck. 

I'm think I'm having trouble getting my head around the fact we have PDF doc and pages. We have Langchain-derived metadata (page_content and page_number) and a seaprate dataframe with most of the rest of metadata. There's a bunch of iterations between and among these things and I'm not sure how to do it.

Here's what we've got:
- We have two fully defined schemas above.
- We have a library catalog containing all the metadata at the PDF document level and most of the PDF page level (page_content and page_number excluded). It contains a unique ID for each PDF called document_id. PAthe is defined above: library_catalog_directory = "../docs/library_catalog/"
- We have code below that pulls in the most recent library catalog into a pandas df
- We have starter code below that pulls the library catalog values into the pdfs_collection
- We have a function called utils.compute_doc_id(pdf_path) that computed the document_id in the library catalog so it should produce the same value when used on the PDFs 



In [ ]:
import pandas as pd

catalog_file_path, last_update_date = utils.get_most_recent_filepath_and_date(
    "library_catalog", library_catalog_directory, "xlsx")

try:
    df = pd.read_excel(catalog_file_path)
    print(f"Successfully imported:  {catalog_file_path}")
except Exception as e:
    os.write(
        1, f"Failed to read the most recent library catalog file: {e}\n".encode())

In [ ]:
# starter code that pulls the library catalog into pdfs_collection

import pypdf
from langchain.document_loaders import PyPDFLoader
import utils  # Ensure utils module with compute_doc_id function is imported

pdfs_collection = client.collections.get("PDF_document")
pdf_pages_collection = client.collections.get("PDF_document_page")

def process_pdf(pdf_path, df):
    try:
        # Compute the document_id for the PDF
        document_id = utils.compute_doc_id(pdf_path)

        # Find the metadata row in df that corresponds to this document_id
        pdf_metadata = df[df['document_id'] == document_id]

        if not pdf_metadata.empty:
            # Process the first (and presumably only) row that matches
            properties = pdf_metadata.iloc[0].to_dict()
            uuid = properties.pop('document_id')
            pdfs_collection.data.insert(properties=properties, uuid=uuid) # I think this pulls all the PDF document fields into Weaviate
        else:
            raise ValueError(f"No metadata found for document ID: {document_id}")

        # Process PDF pages
        pages_objects = []
        loader = PyPDFLoader(pdf_path)
        for page in loader.load():
            pages_objects.append(
                wvc.DataObject(
                    properties={
                        # "title": this would come from the dataframe
                        # "publication_number": this would come from the dataframe
                        "content": page.page_content,
                        "page_number": page.metadata["page"],
                    },
                    references={
                        "hasPdfDocument": document_id
                    }
                )
            )
        pdf_pages_collection.data.insert_many(pages_objects)

    except FileNotFoundError:
        print(f"Error: Could not find {pdf_path}")
    except Exception as e:
        print(f"An error occurred: {e}")


### Option B: Load with PDF's embedded metadata

In [ ]:
import pypdf
from langchain.document_loaders import PyPDFLoader

pdfs_collection = client.collections.get("PDF_document")
pdf_pages_collection = client.collections.get("PDF_document_page")


def process_pdf(pdf_path):
    try:
        reader = pypdf.PdfReader(pdf_path)
        metadata_object = reader.metadata

        document_id = generate_uuid5(metadata_object["/DocId"], uuid.NAMESPACE_DNS)

        pdfs_collection.data.insert(
            uuid=document_id,
            properties={
                "title": metadata_object["/Title"],
                "page_count": int(metadata_object["/PageCount"]),
                # "effective_date": metadata_object["/EffectiveDate"], -> requires date conversion
                "publication_number": metadata_object["/PublicationNumber"],
                "curator": metadata_object["/Curator"],
                # ... need to add more properties
            }
        )

        pages_objects = []
        loader = PyPDFLoader(pdf_path)
        for page in loader.load():
            pages_objects.append(
                wvc.DataObject(
                    properties={
                        "title": metadata_object["/Title"],
                        "content": page.page_content,
                        "page_number": page.metadata["page"],
                        "publication_number": metadata_object["/PublicationNumber"],
                    },
                    references={
                        "hasPdfDocument": document_id
                    }
                )
            )
        pdf_pages_collection.data.insert_many(pages_objects)

    except FileNotFoundError:
        f"Error: Could not find {pdf_path}"


print("Loading PDFs from directory...")

for folder_name, sub_folders, filenames in os.walk(pdf_source_directory):
    for file in filenames:
        if file.lower().endswith('.pdf'):
            process_pdf(os.path.join(folder_name, file))